In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_rows=5

In [20]:
df = pd.read_table('./dataset/val.txt',names=['category','theme','URL','content'],encoding='utf-8')
df = df.dropna()
df.head()

,category,theme,URL,content
0,汽车,新辉腾 ４．２ Ｖ８ ４座加长Ｉｎｄｉｖｉｄｕａｌ版２０１１款 最新报价,http://auto.data.people.com.cn/model_15782/,经销商 电话 试驾／订车Ｕ憬杭州滨江区江陵路１７８０号４００８－１１２２３３转５８６４＃保常...
1,汽车,９１８ Ｓｐｙｄｅｒ概念车,http://auto.data.people.com.cn/prdview_165423....,呼叫热线 ４００８－１００－３００ 服务邮箱 ｋｆ＠ｐｅｏｐｌｅｄａｉｌｙ．ｃｏｍ．ｃｎ
2,汽车,日内瓦亮相 ＭＩＮＩ性能版／概念车－１．６Ｔ引擎,http://auto.data.people.com.cn/news/story_5249...,ＭＩＮＩ品牌在二月曾经公布了最新的ＭＩＮＩ新概念车Ｃｌｕｂｖａｎ效果图，不过现在在日内瓦车展...
3,汽车,清仓大甩卖一汽夏利Ｎ５威志Ｖ２低至３．３９万,http://auto.data.people.com.cn/news/story_6144...,清仓大甩卖！一汽夏利Ｎ５、威志Ｖ２低至３．３９万＝日，启新中国一汽强势推出一汽夏利Ｎ５、威志...
4,汽车,大众敞篷家族新成员 高尔夫敞篷版实拍,http://auto.data.people.com.cn/news/story_5686...,在今年３月的日内瓦车展上，我们见到了高尔夫家族的新成员，高尔夫敞篷版，这款全新敞篷车受到了众...


In [21]:
print('总的新闻条目：%s'%df.shape[0])

总的新闻条目：5000


In [22]:
print('总的新闻类别:%s' %len(set(df['category'].values.tolist())))

总的新闻类别:10


In [23]:
categorys=list(set(df['category'].values.tolist()))
category_df=pd.DataFrame(categorys)
category_df

,0
0,科技
1,汽车
...,...
8,财经
9,教育


- 统计各个类别的分布情况

In [24]:
caregorys_num=[]
for i in categorys:
    caregorys_num.append(df[df['category']==i].shape[0])
pd.DataFrame([categorys,caregorys_num])

,0,1,2,3,4,5,6,7,8,9
0,科技,汽车,文化,时尚,军事,娱乐,健康,体育,财经,教育
1,500,500,500,500,500,500,500,500,500,500


- 数据集切分

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
x_train, x_test, y_train, y_test = train_test_split(df['content'], 
                                                    df['category'], 
                                                    random_state=1)

In [27]:
print('Number of rows in the total set:{}'.format(df.shape[0]))
print('Number of rows in the training set:{}'.format(x_train.shape[0]))
print('Number of rows in the test set: {}'.format(x_test.shape[0]))

Number of rows in the total set:5000
Number of rows in the training set:3750
Number of rows in the test set: 1250


- 删除停用词

In [28]:
stopwords = pd.read_csv('./dataset/stopwords.txt',index_col=False,sep='\t',quoting=3,names=['stopword'],encoding='utf-8')
stopwords = stopwords.stopword.values.tolist()

- 分词

In [29]:
import jieba 

In [30]:
cw=lambda x:list(jieba.lcut(x))
df['words']=df['content'].apply(cw)

- 提取关键词

In [31]:
import jieba.analyse
index = 2400
print(" ".join(jieba.analyse.extract_tags(' '.join(df['words'][index]),topK=5,withWeight = False)))

耐克 阿迪达斯 欧洲杯 球衣 西班牙


In [32]:
label_mapping = {"汽车": 1, "财经": 2, "科技": 3, "健康": 4, "体育":5, "教育": 6,"文化": 7,"军事": 8,"娱乐": 9,"时尚": 0}

In [123]:
x_train

3186    中新网上海６月１４日电（记者于俊）今年的父亲节，人们可通过网络“吃”上一顿“电影快餐”。微电...
632     目前正值中报发布之际，多家券商认为，上半年银行整体利润增速下滑是大概率事件。国泰君安报告预计...
                              ...                        
3980    中国航天员中心揭秘“神九”航天员衣食居住与娱乐４樱保度胀碜既啡牍欤到１７日傍晚顺利完成３次变...
235     【网上车市绍兴站讯】普力马作为海马汽车旗下一款入门级ＭＰＶ，凭借着成熟的技术以及高性价比赢得...
Name: content, Length: 3750, dtype: object

In [34]:
y_train

3186    文化
632     财经
        ..
3980    军事
235     汽车
Name: category, Length: 3750, dtype: object

In [36]:
y_train_map=y_train.map(label_mapping)
y_train_map

3186    7
632     2
       ..
3980    8
235     1
Name: category, Length: 3750, dtype: int64

In [80]:
df_x_train=pd.DataFrame({'x_train':x_train})
x_training=df_x_train.x_train.apply(cw).values.tolist()

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

In [101]:
cv = CountVectorizer(analyzer="char",lowercase=False)

In [130]:
training_data=[]
for i in range(len(x_train)):
    training_data.append(cv.fit_transform(x_training[i])
print(training_data)

[<453x346 sparse matrix of type '<class 'numpy.int64'>'
	with 677 stored elements in Compressed Sparse Row format>, <292x221 sparse matrix of type '<class 'numpy.int64'>'
	with 479 stored elements in Compressed Sparse Row format>, <643x409 sparse matrix of type '<class 'numpy.int64'>'
	with 1098 stored elements in Compressed Sparse Row format>, <746x387 sparse matrix of type '<class 'numpy.int64'>'
	with 1186 stored elements in Compressed Sparse Row format>, <535x329 sparse matrix of type '<class 'numpy.int64'>'
	with 857 stored elements in Compressed Sparse Row format>, <696x275 sparse matrix of type '<class 'numpy.int64'>'
	with 1040 stored elements in Compressed Sparse Row format>, <161x109 sparse matrix of type '<class 'numpy.int64'>'
	with 254 stored elements in Compressed Sparse Row format>, <503x314 sparse matrix of type '<class 'numpy.int64'>'
	with 739 stored elements in Compressed Sparse Row format>, <308x299 sparse matrix of type '<class 'numpy.int64'>'
	with 495 stored elem

In [117]:
cv.get_feature_names()

['β',
 '、',
 '。',
 '【',
 '】',
 '一',
 '三',
 '上',
 '下',
 '不',
 '与',
 '两',
 '个',
 '中',
 '为',
 '主',
 '之',
 '也',
 '买',
 '乱',
 '了',
 '亮',
 '从',
 '代',
 '以',
 '仪',
 '价',
 '优',
 '但',
 '体',
 '作',
 '侧',
 '俊',
 '借',
 '做',
 '储',
 '元',
 '充',
 '入',
 '全',
 '六',
 '兴',
 '其',
 '具',
 '内',
 '凭',
 '出',
 '分',
 '则',
 '刚',
 '到',
 '制',
 '刻',
 '力',
 '功',
 '动',
 '匹',
 '十',
 '升',
 '原',
 '及',
 '友',
 '发',
 '变',
 '可',
 '台',
 '后',
 '和',
 '哪',
 '啵',
 '嗉',
 '围',
 '在',
 '型',
 '埃',
 '外',
 '大',
 '存',
 '定',
 '宜',
 '家',
 '富',
 '寸',
 '将',
 '少',
 '尚',
 '尺',
 '局',
 '工',
 '市',
 '布',
 '幅',
 '并',
 '店',
 '度',
 '庭',
 '式',
 '引',
 '强',
 '当',
 '形',
 '彰',
 '很',
 '律',
 '得',
 '态',
 '性',
 '情',
 '惠',
 '感',
 '成',
 '户',
 '手',
 '扭',
 '扰',
 '技',
 '拥',
 '择',
 '指',
 '控',
 '提',
 '搭',
 '擎',
 '效',
 '整',
 '新',
 '方',
 '旗',
 '无',
 '日',
 '时',
 '是',
 '显',
 '普',
 '景',
 '更',
 '最',
 '有',
 '朋',
 '术',
 '机',
 '材',
 '条',
 '来',
 '板',
 '格',
 '樟',
 '次',
 '款',
 '毂',
 '毅',
 '比',
 '气',
 '求',
 '汽',
 '洁',
 '流',
 '海',
 '淮',
 '淼',
 '漂',
 '炫',
 '炮',
 '点',
 '熟',
 '物',
 '特'

In [125]:
for i in range(len(x_train)):
    train_array=cv.transform(x_training[i]).toarray()

In [114]:
frequency_matrix = pd.DataFrame(train_array, columns = cv.get_feature_names())
frequency_matrix

,β,、,。,【,】,一,三,上,下,不,...,Ｎ,Ｐ,Ｓ,Ｔ,Ｖ,ｋ,ｍ,ｐ,ｒ,ｗ
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
411,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
from sklearn.naive_bayes import MultinomialNB

In [129]:
for i in range(len(x_train)):
    naive_bayes = MultinomialNB()
    naive_bayes.fit(training_data[i], y_train)

ValueError: Found input variables with inconsistent numbers of samples: [453, 3750]